1. m1 + m4 + tabnet

In [10]:
import pandas as pd
import numpy as np

DATA_PATH = "../../data/raw/"
SUB_PATH = "./submissions/"

# 파일 로드
nn = pd.read_csv(f"{SUB_PATH}0130-1923.csv")['voted']  # 0.7811
snn = pd.read_csv(f"{SUB_PATH}Diversity_Focal_SNN_AUC_0.7708.csv")['voted']  # SNN
nn_tabnet = pd.read_csv(f"{SUB_PATH}Final_Ensemble_96%_4%_AUC0.78046.csv")['voted']  # nn+TabNet

# TabNet 역산 (nn_tabnet = nn*0.96 + TabNet*0.04)
tabnet = (nn_tabnet - nn * 0.96) / 0.04

print("="*60)
print("nn + SNN + TabNet 동시 투입")
print("="*60)

# 상관계수
print(f"nn vs SNN:    {nn.corr(snn):.4f}")
print(f"nn vs TabNet: {nn.corr(tabnet):.4f}")
print(f"SNN vs TabNet: {snn.corr(tabnet):.4f}\n")

# 후보 조합 (nn : SNN : TabNet)
candidates = [
    (0.90, 0.05, 0.05),
    (0.88, 0.06, 0.06),
    (0.92, 0.04, 0.04),
    (0.89, 0.05, 0.06),
    (0.91, 0.04, 0.05),
    (0.89, 0.06, 0.05),
    (0.88, 0.05, 0.07),
]

results = []
for w1, w2, w3 in candidates:
    blend = nn * w1 + snn * w2 + tabnet * w3
    expected = 0.7811 * w1 + 0.7708 * w2 + 0.7812 * w3  # 근사치
    
    results.append({
        'nn': w1,
        'snn': w2,
        'tabnet': w3,
        'expected': expected
    })
    
    print(f"{w1:.2f} : {w2:.2f} : {w3:.2f} → 예상: {expected:.5f}")

df = pd.DataFrame(results).sort_values('expected', ascending=False)

print("\n" + "="*60)
print("Top 3 조합")
print("="*60)
print(df.head(3).to_string(index=False))

print("\nTop 3 저장하시겠습니까? (y/n)")

nn + SNN + TabNet 동시 투입
nn vs SNN:    0.9751
nn vs TabNet: 0.9883
SNN vs TabNet: 0.9634

0.90 : 0.05 : 0.05 → 예상: 0.78059
0.88 : 0.06 : 0.06 → 예상: 0.78049
0.92 : 0.04 : 0.04 → 예상: 0.78069
0.89 : 0.05 : 0.06 → 예상: 0.78059
0.91 : 0.04 : 0.05 → 예상: 0.78069
0.89 : 0.06 : 0.05 → 예상: 0.78049
0.88 : 0.05 : 0.07 → 예상: 0.78059

Top 3 조합
  nn  snn  tabnet  expected
0.91 0.04    0.05  0.780693
0.92 0.04    0.04  0.780692
0.88 0.05    0.07  0.780592

Top 3 저장하시겠습니까? (y/n)


2. m1 + (m1+tabnet) 비율 테스트

In [8]:
import pandas as pd
import numpy as np

DATA_PATH = "../../data/raw/"
SUB_PATH = "./submissions/"

# 파일 로드
m1 = pd.read_csv(f"{SUB_PATH}0130-1923.csv")['voted']  
m1_tabnet = pd.read_csv(f"{SUB_PATH}Final_Ensemble_96%_4%_AUC0.78046.csv")['voted']

# TabNet 역산 (m1_tabnet = m1*0.96 + TabNet*0.04)
# TabNet = (m1_tabnet - m1*0.96) / 0.04
tabnet_pure = (m1_tabnet - m1 * 0.96) / 0.04

print("="*60)
print("m1 + TabNet 미세 조정 (96:4 주변)")
print("="*60)

candidates = [
    0.97, 0.965, 0.96, 0.955, 0.95, 0.945, 0.94
]

results = []
for w in candidates:
    blend = m1 * w + tabnet_pure * (1-w)
    expected = 0.78116 * w + 0.78046 * (1-w)  # 근사치
    
    results.append({
        'm1_weight': w,
        'tabnet_weight': 1-w,
        'expected_auc': expected
    })
    
    print(f"{w:.3f} : {1-w:.3f} → 예상AUC: {expected:.5f}")

df_results = pd.DataFrame(results)
df_results = df_results.sort_values('expected_auc', ascending=False)

print("\n" + "="*60)
print("Top 3")
print("="*60)
print(df_results.head(3).to_string(index=False))

print("\n저장할 조합 선택:")
for i, row in df_results.head(5).iterrows():
    print(f"{i+1}. {row['m1_weight']:.3f}:{row['tabnet_weight']:.3f}")

m1 + TabNet 미세 조정 (96:4 주변)
0.970 : 0.030 → 예상AUC: 0.78114
0.965 : 0.035 → 예상AUC: 0.78114
0.960 : 0.040 → 예상AUC: 0.78113
0.955 : 0.045 → 예상AUC: 0.78113
0.950 : 0.050 → 예상AUC: 0.78112
0.945 : 0.055 → 예상AUC: 0.78112
0.940 : 0.060 → 예상AUC: 0.78112

Top 3
 m1_weight  tabnet_weight  expected_auc
     0.970          0.030      0.781139
     0.965          0.035      0.781135
     0.960          0.040      0.781132

저장할 조합 선택:
1. 0.970:0.030
2. 0.965:0.035
3. 0.960:0.040
4. 0.955:0.045
5. 0.950:0.050


In [11]:
# Top 3 저장
save_candidates = [
    (0.970, 0.030),  # 최고
    (0.965, 0.035),
    (0.955, 0.045),  # 조금 더 공격적
]

for w1, w2 in save_candidates:
    blend = nn * w1 + tabnet * w2
    final_sub = pd.read_csv(f"{DATA_PATH}sample_submission.csv")
    final_sub['voted'] = blend
    filename = f"{SUB_PATH}Final_{w1:.1%}nn_{w2:.1%}tab.csv"
    final_sub.to_csv(filename, index=False)
    print(f"✅ {filename}")

print("\n🎯 Top 3 저장 완료!")
print("제출 순서:")
print("1. Final_97.0%nn_3.0%tab.csv  (예상 최고)")
print("2. Final_96.5%nn_3.5%tab.csv")
print("3. Final_95.5%nn_4.5%tab.csv")

✅ ./submissions/Final_97.0%nn_3.0%tab.csv
✅ ./submissions/Final_96.5%nn_3.5%tab.csv
✅ ./submissions/Final_95.5%nn_4.5%tab.csv

🎯 Top 3 저장 완료!
제출 순서:
1. Final_97.0%nn_3.0%tab.csv  (예상 최고)
2. Final_96.5%nn_3.5%tab.csv
3. Final_95.5%nn_4.5%tab.csv


1번 제출 보류

3. 기존 FT+MLP랑도 앙상블 테스트

In [16]:
import pandas as pd
import numpy as np

DATA_PATH = "../../data/raw/"
SUB_PATH = "./submissions/"

# 파일 로드
nn = pd.read_csv(f"{SUB_PATH}0130-1923.csv")['voted']  # 0.7811
nn_tabnet = pd.read_csv(f"{SUB_PATH}Final_Ensemble_96%_4%_AUC0.78046.csv")['voted']
ft_mlp = pd.read_csv(f"{SUB_PATH}submission_09_Ensemble_MLP_FT_v1.csv")['voted']  # 0.778

# TabNet 역산
tabnet = (nn_tabnet - nn * 0.96) / 0.04

print("="*60)
print("상관계수 확인")
print("="*60)
print(f"nn vs TabNet:  {nn.corr(tabnet):.4f}")
print(f"nn vs FT+MLP:  {nn.corr(ft_mlp):.4f}")
print(f"TabNet vs FT+MLP: {tabnet.corr(ft_mlp):.4f}\n")

print("="*60)
print("3-Model: nn + TabNet + FT+MLP")
print("="*60)

# 후보 (nn : TabNet : FT+MLP)
candidates = [
    (0.94, 0.03, 0.03),
    (0.93, 0.04, 0.03),
    (0.95, 0.03, 0.02),
    (0.92, 0.05, 0.03),
    (0.93, 0.05, 0.02),
    (0.94, 0.04, 0.02),
    (0.91, 0.06, 0.03),
    (0.92, 0.04, 0.04),
]

results = []
for w1, w2, w3 in candidates:
    blend = nn * w1 + tabnet * w2 + ft_mlp * w3
    expected = 0.7811 * w1 + 0.7812 * w2 + 0.778 * w3
    
    results.append({
        'nn': w1,
        'tabnet': w2,
        'ft_mlp': w3,
        'expected': expected
    })
    
    print(f"{w1:.2f} : {w2:.2f} : {w3:.2f} → 예상: {expected:.5f}")

df = pd.DataFrame(results).sort_values('expected', ascending=False)

print("\n" + "="*60)
print("Top 3 조합")
print("="*60)
print(df.head(3).to_string(index=False))

print("\n" + "="*60)
print("비교")
print("="*60)
print(f"현재 최고 (nn 96% + TabNet 4%):     0.78120")
print(f"97:3 조합:                         0.78114")
print(f"3-Model 최고:                      {df.iloc[0]['expected']:.5f}")

상관계수 확인
nn vs TabNet:  0.9883
nn vs FT+MLP:  0.9826
TabNet vs FT+MLP: 0.9832

3-Model: nn + TabNet + FT+MLP
0.94 : 0.03 : 0.03 → 예상: 0.78101
0.93 : 0.04 : 0.03 → 예상: 0.78101
0.95 : 0.03 : 0.02 → 예상: 0.78104
0.92 : 0.05 : 0.03 → 예상: 0.78101
0.93 : 0.05 : 0.02 → 예상: 0.78104
0.94 : 0.04 : 0.02 → 예상: 0.78104
0.91 : 0.06 : 0.03 → 예상: 0.78101
0.92 : 0.04 : 0.04 → 예상: 0.78098

Top 3 조합
  nn  tabnet  ft_mlp  expected
0.93    0.05    0.02  0.781043
0.94    0.04    0.02  0.781042
0.95    0.03    0.02  0.781041

비교
현재 최고 (nn 96% + TabNet 4%):     0.78120
97:3 조합:                         0.78114
3-Model 최고:                      0.78104
